<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/Coil3D_Frenet_Serret_Transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import jax.numpy as jnp
import jax
from plotly.subplots import make_subplots

In [10]:


def curve(t,R, mz):
    xyz= jnp.array([R*jnp.cos(t), R*jnp.sin(t), jnp.exp(t/5)*mz/(2*np.pi)])
    return xyz


#Research on what is the Jacobian of a function.
#How does this differ from the gradient of a function?  
curve_jac=jax.jacobian(curve, 0)
def tangent(t,R,mz):
    v=curve_jac(t,R,mz)
    return v/jnp.linalg.norm(v)
curve_normal=jax.jacobian(tangent, 0)  #Frenet-Serret for normal
print(curve(np.pi/6.,1., 2*np.pi))
print(curve_jac(np.pi/6.,1., 2*np.pi))
print(curve_normal(np.pi/6.,1., 2*np.pi))

[0.8660254 0.5       1.1103994]
[-0.5         0.8660254   0.22207987]
[-0.84083986 -0.4960555   0.04132165]


In [11]:
def surf1(t,theta,r,R,mz):

    vl = curve(t,R,mz)
    v_jac = curve_jac(t,R,mz)

    #What are n1 and n2?
    #the choice of unit vector in z-direction leads to one normal always perpendicular to z
    #no difference for circle, but noticeable for ellipse cross-section
    n1=-jnp.cross(v_jac,jnp.array([0.,0.,1.]))
    n1=n1/jnp.linalg.norm(n1)
    n2=jnp.cross(n1,v_jac)
    n2=0.5*n2/jnp.linalg.norm(n2)

    #rotation matrix
    rot = jnp.array([[jnp.cos(theta), -jnp.sin(theta)],[jnp.sin(theta),jnp.cos(theta)]])

    vx = jnp.array([r,0])   #r along the x-coordinate
    vrot = jnp.matmul(rot, vx)  #rotate by theta
    vsurf = jnp.matmul(jnp.stack([n1,n2], axis=1),vrot)  #what is this???

    return vl+vsurf  #why add?

In [12]:
def surf2(t,theta,r,R,mz):

    vl = curve(t,R,mz)
    v_jac = curve_jac(t,R,mz)

    #Frenet-Serret
    #acceleration vector for n1
    #the magnitude of n1 is the acceleration with respect to t and important
    #if we were not just doing this to generate an elliptical cross-section
    n1=curve_normal(t,R,mz)
    n1=n1/jnp.linalg.norm(n1)
    n2=jnp.cross(n1,v_jac)
    n2=0.5*n2/jnp.linalg.norm(n2)

    #rotation matrix
    rot = jnp.array([[jnp.cos(theta), -jnp.sin(theta)],[jnp.sin(theta),jnp.cos(theta)]])

    vx = jnp.array([r,0])   #r along the x-coordinate
    vrot = jnp.matmul(rot, vx)  #rotate by theta
    vsurf = jnp.matmul(jnp.stack([n1,n2], axis=1),vrot)  #what is this???

    return vl+vsurf  #why add?

In [13]:
t=jnp.linspace(0,4*np.pi,120)
theta=jnp.linspace(0,2*np.pi,16)
R=1.
r=0.2
mz=0.8

surf1_v = jnp.vectorize(surf1, signature='(),(),(),(),()->(3)')
s1=surf1_v(t[:,None],theta[None,:],r, R,mz)

surf2_v = jnp.vectorize(surf2, signature='(),(),(),(),()->(3)')
s2=surf2_v(t[:,None],theta[None,:],r, R,mz)

jnp.sum(jnp.abs(s1-s2))

DeviceArray(9.478931, dtype=float32)

In [14]:
fig = make_subplots(rows=1,cols=1,specs=[[{'type': 'scene'}]])

for i in range(t.size):
    x,y,z = s1[i,:].T  #why tranpose?  take a look at unpacking for small matrices
    fig.add_scatter3d(x=x,y=y,z=z,mode='lines', line_color='white', row=1,col=1)

for i in range(theta.size):
    x,y,z = s1[:,i].T
    fig.add_scatter3d(x=x,y=y,z=z,mode='lines', line_color='white', row=1,col=1)

for i in range(t.size):
    x,y,z = s2[i,:].T
    fig.add_scatter3d(x=x,y=y,z=z,mode='lines', line_color='red', row=1,col=1)

for i in range(theta.size):
    x,y,z = s2[:,i].T
    fig.add_scatter3d(x=x,y=y,z=z,mode='lines', line_color='red', row=1,col=1)



fig.update_layout(width=800,template='plotly_dark',showlegend=False, scene = dict(
                        yaxis = dict(range=[-2,2]),
                        zaxis = dict(range=[-1,2])))

In [ ]:
#what is the purpose of (0,1)
surf_jac=jax.jacobian(surf,(0,1))

NameError: ignored

In [ ]:
#examine different t, theta (multiples of pi/2) values and for convenient r, R and also mz=0 
surf(0.,0.,r, R, 0.8)

DeviceArray([1.2, 0. , 0. ], dtype=float32)

In [ ]:
#In the Jacobian, why does 1.2 being equal to R+r and 0.1975 being close to r=0.2 make sense?
surf_jac(0.,0.,r,R,1)

(DeviceArray([0.        , 1.2       , 0.15915494], dtype=float32, weak_type=True),
 DeviceArray([ 0.        , -0.03143534,  0.1975141 ], dtype=float32, weak_type=True))

In [ ]:
nt=120
ntheta=64
t=jnp.linspace(0,4*np.pi,nt,endpoint=False)
theta=jnp.linspace(0,2*np.pi,ntheta,endpoint=False)

dt=4*np.pi/nt
dtheta=2*np.pi/ntheta

def surf_jacdet(t,theta,r):

    #why name these dxyz_dt and dxyz_dtheta?
    dxyz_dt, dxyz_dtheta = surf_jac(t,theta,r,R,mz)

    #why do we multiply by dt and dtheta?
    #recall the geometric meaning of the cross product
    return jnp.linalg.norm(jnp.cross(dxyz_dt*dt, dxyz_dtheta*dtheta))

surf_jacdet_vec=jnp.vectorize(surf_jacdet,signature='(),(),()->()')

jnp.sum(surf_jacdet_vec(t[:,None],theta[None,:],r))

DeviceArray(15.918863, dtype=float32)

In [ ]:
#arc length of curve, what approximation are we making here for the area?
curve_jac_vec=jnp.vectorize(curve_jac,signature='(),(),()->(3)')

#why do we take the sqrt and sum of squares?
arclength=jnp.sum(jnp.sqrt(jnp.sum(curve_jac_vec(t,R,mz)**2,axis=1)))*dt
arclength*2*np.pi*r


DeviceArray(15.918854, dtype=float32)